In [1]:
import speech_recognition as sr
import pyttsx3
import re
import datetime

In [2]:
recognizer = sr.Recognizer()
engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

In [3]:
def speak(audio):
    engine.say(audio)
    engine.runAndWait()

In [4]:
def listen():
    with sr.Microphone() as source:
        print("Sleeping...")
        recognizer.adjust_for_ambient_noise(source)
        recognizer.energy_threshold = 4000
        recognizer.pause_threshold = 1

        audio = recognizer.listen(source)
        spoken_text = recognizer.recognize_google(audio).lower()
        if "assistant activate" in spoken_text:
            print("Hey, how can I help you?")
            speak("Hey, how can I help you?")
            while True:
                audio = recognizer.listen(source)
                try:
                    spoken_text = recognizer.recognize_google(audio).lower()
                    print("Heard:", spoken_text)

                    if any(word in spoken_text for word in ["exit", "bye", "see ya"]):
                        speak("Exiting...")
                        break
                    elif "set a reminder" in spoken_text:
                        reminder_details = extract_reminder_details(spoken_text)
                        if reminder_details:
                            print(reminder_details)
                            #set_reminder(reminder_details)
                            speak("Reminder set successfully.")
                            break
                        
                except sr.UnknownValueError:
                    pass

                except sr.RequestError as e:
                    print(f"Could not request results; {e}")

def extract_reminder_details(spoken_text):
    pattern = r"(tomorrow|today|everyday|on \w+ \d+)(?: at (\d+))?"

    match = re.search(pattern, spoken_text)
    if match:
        date_str, time_str = match.groups()
        date = parse_date(date_str)
        time = parse_time(time_str) if time_str else None

        content = spoken_text.split("reminder for a ", 1)[-1]

        reminder_details = {
            "date": date,
            "time": time,
            "content": content
        }
        return reminder_details
    else:
        print("Reminder details not found.")
        return None

def parse_date(date_str):
    if "tomorrow" in date_str:
        tomorrow = datetime.date.today() + datetime.timedelta(days=1)
        return tomorrow.strftime("%Y-%m-%d")
    elif "today" in date_str:
        return datetime.date.today().strftime("%Y-%m-%d")
    else:
        return None

def parse_time(time_str):
    if time_str:
        return f"{time_str}:00 AM"
    else:
        return None

In [5]:
listen()

Sleeping...
Hey, how can I help you?
Heard: set a reminder to water my plants everyday
Reminder details not found.
Heard: exit
